In [ ]:
# Using OpenCv for Gesture Recognition and similar utilities
import cv2

# np as alias using it for numpy array operatiions
import numpy as np

# Using pynput library and using it's mouse functionality for binding
from pynput.mouse import Button, Controller

# For control of display and getting the screen resolution
import wx

# Using mouse package utility and getting the control as variable mouse
mouse=Controller()

# Setting up the wx environment for getting display
app=wx.App(False)

# Storing tuple of screen resolution in sx and sy co-ordinate variables
(sx,sy)=wx.GetDisplaySize()

# Setting manual webcam screen display window size
(camx,camy)=(320,240)

# For color segmentation using a range of green colors to identify the gesture
lowerBound=np.array([33,80,40])
upperBound=np.array([102,255,255])

# Capturing the video and storing to cam
cam= cv2.VideoCapture(0)

# Utilities for applying smoothening filters
kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))

# Tracking mouse locations for smooth movements in numpy arrays
mLocOld = np.array([0,0])
mouseLoc = np.array([0,0])

# Flag to know the switching between the state gestures.
pinchFlag=0

#Initialising window size of combined 2 gestures
openx, openy, openw, openh =(0,0,0,0)

#Damping factor for taking average  of mouse locations
DampingFactor = 2

#Loop to get the number of frames from the video and do operations on them
while True:
    # Capturing the video frame
    ret, img=cam.read()
    
    #Resizing it to desired size
    img=cv2.resize(img,(camx,camy))

    #convert BGR to HSV
    imgHSV= cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    
    # create the Mask
    mask=cv2.inRange(imgHSV,lowerBound,upperBound)
    
    #Applying morphological operations on binary
    maskOpen=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernelOpen)
    maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)
    
    #Selecting the closed mask for masking green color
    maskFinal=maskClose
    
    # Calculating total number of contours
    conts,h=cv2.findContours(maskFinal.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    # Working logic based on number of gestures recognised
    if(len(conts)==2):
        #Checking for gesture state for switching the logic
        if(pinchFlag==1):
            pinchFlag=0
            # If state is default removing the left click
            mouse.release(Button.left)
        
        # Storing the states of co-ordinates
        x1,y1,w1,h1=cv2.boundingRect(conts[0])
        x2,y2,w2,h2=cv2.boundingRect(conts[1])
        
        # Capturing them in rectangles of blue color
        cv2.rectangle(img,(x1,y1),(x1+w1,y1+h1),(255,0,0),2)
        cv2.rectangle(img,(x2,y2),(x2+w2,y2+h2),(255,0,0),2)
        
        # Taking out the average of the cordinates to find the centre of rectangle
        cx1=x1+w1/2
        cy1=y1+h1/2
        cx2=x2+w2/2
        cy2=y2+h2/2
        
        # Taking out the average of the cordinates to find the centre of the line joining the centres.
        cx=(cx1+cx2)/2
        cy=(cy1+cy2)/2
        
        # Drawing the line of blue color between the centres of rectangle
        cv2.line(img, (cx1,cy1),(cx2,cy2),(255,0,0),2)
        
        #Drawing a bullet circle on the centre of the line for mouse binding
        cv2.circle(img, (cx,cy),2,(0,0,255),2)
        
        # Changing of mouse cursor's new location based on the previous cordinated of cursor
        mouseLoc = mLocOld + ((cx,cy)-mLocOld)/DampingFactor
        
        # Alloting new position to the mouse cursor on the weighted average of cordinates
        mouse.position=(sx-(mouseLoc[0]*sx/camx),mouseLoc[1]*sy/camy)
        
        # Pasisng the logic if condition does not satisfy
        while mouse.position!=(sx-(mouseLoc[0]*sx/camx), mouseLoc[1]*sy/camy):
            pass
        
        #Storing the old mouse location for next usage.
        mLocOld = mouseLoc
        
        # Calculating the bigger rectangle of combined gestures for area estimation
        openx, openy, openw, openh = cv2.boundingRect(np.array([[[x1,y1],[x1+w1,y1+h1],[x2,y2],[x2+w2,y2+h2]]]))
        
        # Uncomment below to check the bigger rectangle of combined gestures
        #cv2.rectangle(img,(openx,openy),(openx+openw,openy+openh),(255,0,0),2)
        
        # Clicking operations if the objects merges to one
    elif(len(conts)==1):
        # Bounding rectangle on the single object created
        x,y,w,h=cv2.boundingRect(conts[0])
        
        #Checking the flag condition
        if(pinchFlag==0):
            # Checking if one object goes out of screen what should be difference in the ratios so that logic applies
            if(abs((w*h - openw*openh)*100/(w*h)) < 30):
                pinchFlag=1
                
                #Click the left button
                mouse.press(Button.left)
                openx, openy, openw, openh =(0,0,0,0)
        else:
            #Otherwise if closing to opening the gesture then don't press
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            cx=x+w/2
            cy=y+h/2
            cv2.circle(img,(cx,cy),(w+h)/4,(0,0,255),2)
            
            # Store the mouse location
            mouseLoc = mLocOld + ((cx,cy)-mLocOld)/DampingFactor
            mouse.position=(sx-(mouseLoc[0]*sx/camx), mouseLoc[1]*sy/camy)
            
            # Checking again
            while mouse.position!=(sx-(mouseLoc[0]*sx/camx), mouseLoc[1]*sy/camy):
                pass
            mLocOld = mouseLoc
            
    # Showing on webcam
    cv2.imshow("cam",img)
    
    # Waiting for frames on the basis of ticks.
    cv2.waitKey(5)